# Tutorial: Approximate Local Text Explanation (ALTE) - IMDb movie review sentiment explanation
Explaining an IMDb Movie Reviews Text Classification Tensorflow Model localy for one datapoint with Approximate Local Text Explanation via linear models. This is a local explanation procedure in which a single input data point is analyzed. The components of this input data point (token) are activated or deactivated by permutations of a binary vector of the same size as the number of components of the input data point. All permutations are classified by the original classification model and stored in a meta dataset. This meta dataset is then used to train a linear classification model, thus linearly approximating the original classification function.

### 1. Imports and Configuration
For the original classification model we use TensorFlow, for the dataset and its transformation TensorFlow Dataset and Pandas Dataframe, for mathemathical operations Numpy and text processing RegEx and String libraries.

In [1]:
import re
import string
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
import pandas as pd

from tensorflow.python.keras import Sequential
from tensorflow.python.keras.layers import Embedding, Dropout, GlobalAveragePooling1D, Dense
from tensorflow.python.keras.losses import BinaryCrossentropy
from tensorflow.python.keras.metrics import BinaryAccuracy
from ate.base import ALTE, ATE_Options

Configuration by TensorFlow tutorial "Basic Text Classification":
- MAX_FEATURES = Vocabulary size
- EMBEDDING_DIM = Token embedding vector size
- SEQUENCE_LENGTH = Max. tokens for classification
- BATCH_SIZE = Dataset batching size
- AUTOTUNE = Buffer size

In [2]:
MAX_FEATURES = 10000
EMBEDDING_DIM = 16
SEQUENCE_LENGTH = 250
BATCH_SIZE = 32
AUTOTUNE = tf.data.AUTOTUNE

### 2. Loading and Preparation
Initial downloading of IMDb movie review dataset by usind TensorFlow Dataset.

In [3]:
raw_train_ds = tfds.load('imdb_reviews', split='train').batch(BATCH_SIZE)
raw_test_ds = tfds.load('imdb_reviews', split='test').batch(BATCH_SIZE)

Metal device set to: Apple M1 Pro


2023-03-25 12:50:26.134844: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-03-25 12:50:26.134933: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Using a **custom standardization** function to process the textual data and stripping html tags and punctuation.

In [4]:
def custom_standardization(input_data):
    lowercase = tf.strings.lower(input_data)
    stripped_html = tf.strings.regex_replace(lowercase, '<br />', ' ')
    return tf.strings.regex_replace(stripped_html, '[%s]' % re.escape(string.punctuation), '')

**Vectorization** of the textual data by using a vocabulary and transforming the tokens into integer ids (id vectors).

In [5]:
vectorize_layer = tf.keras.layers.TextVectorization(
    standardize=custom_standardization,
    max_tokens=MAX_FEATURES,
    output_mode='int',
    output_sequence_length=SEQUENCE_LENGTH)

In [6]:
train_text = raw_train_ds.map(lambda x: x['text'])
vectorize_layer.adapt(train_text)

2023-03-25 12:50:26.208880: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2023-03-25 12:50:26.255362: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


In [7]:
def vectorize_text(text, label):
    text = tf.expand_dims(text, -1)
    return vectorize_layer(text), label

In [8]:
train_ds = raw_train_ds.map(lambda x: vectorize_text(x['text'], x['label']))
test_ds = raw_test_ds.map(lambda x: vectorize_text(x['text'], x['label']))

In [9]:
train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
test_ds = test_ds.cache().prefetch(buffer_size=AUTOTUNE)

### 3. Creating and Training
Creation and trainig of the classification model.

In [10]:
model = Sequential([
    Embedding(MAX_FEATURES + 1, EMBEDDING_DIM),
    Dropout(0.2),
    GlobalAveragePooling1D(),
    Dropout(0.2),
    Dense(1)
])

In [11]:
model.compile(loss=BinaryCrossentropy(from_logits=True),
              optimizer='adam',
              metrics=BinaryAccuracy(threshold=0.0))

In [12]:
history = model.fit(train_ds, epochs=5)

Epoch 1/5


2023-03-25 12:50:28.258956: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


782/782 [==============================] - 15s 18ms/step - loss: 0.6487 - binary_accuracy: 0.7178
Epoch 2/5
782/782 [==============================] - 10s 13ms/step - loss: 0.5003 - binary_accuracy: 0.8236
Epoch 3/5
782/782 [==============================] - 9s 12ms/step - loss: 0.3931 - binary_accuracy: 0.8628
Epoch 4/5
782/782 [==============================] - 9s 12ms/step - loss: 0.3310 - binary_accuracy: 0.8820
Epoch 5/5
782/782 [==============================] - 9s 12ms/step - loss: 0.2898 - binary_accuracy: 0.8973


In [13]:
loss, accuracy = model.evaluate(test_ds)

print("Loss: ", loss)
print("Accuracy: ", accuracy)

  6/782 [..............................] - ETA: 8s - loss: 0.3746 - binary_accuracy: 0.8125  

2023-03-25 12:51:21.463911: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


782/782 [==============================] - 4s 4ms/step - loss: 0.3368 - binary_accuracy: 0.8637
Loss:  0.33678048849105835
Accuracy:  0.8636800646781921


### 4. Explaining
Initializing ALTE by providing the Tokinization, Vectorization (only needed for post-processing to identify token similarities), Classification Function. Additionaly initializing the ATE_Options with the column names, permutation (approximation) steps, permutation border (per step), classes (1 for binary; >2 for multi-class/-label), linear epochs. 

In [14]:
def tokenize(x):
    return np.array(x.lower().split())
def classify(x):
    id_vecs = []
    for e in x['text'].tolist():
        id_vec = []
        if len(e) > 0:
            id_vec = vectorize_layer(' '.join(e)).numpy().tolist()
        if len(id_vec) == 0:
            id_vec = [0]*SEQUENCE_LENGTH
        id_vecs.append(id_vec)
    return model.predict(id_vecs)
ate = ALTE(
    tokenize,
    lambda x: x, #INFO: Only needed for effect transformation.
    classify,
)
options = ATE_Options(['text'], 5, 10000, 1, 10)

Running the **explanation** on one entry from the dataset.

In [15]:
test_df = tfds.as_dataframe(raw_test_ds).head(1)
test_df = pd.DataFrame([test_df['text'][0][0].decode('UTF-8')], columns=['text'])
effects = ate.explain(test_df, options)

  0%|                                                                                                                                                        | 0/5 [00:00<?, ?it/s]2023-03-25 12:51:26.227891: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


Epoch 1/10


2023-03-25 12:51:26.509800: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


5/5 [==============================] - 1s 65ms/step - loss: 0.1806 - accuracy: 0.4969
Epoch 2/10
5/5 [==============================] - 0s 26ms/step - loss: 0.1763 - accuracy: 0.5031
Epoch 3/10
5/5 [==============================] - 0s 10ms/step - loss: 0.1732 - accuracy: 0.5220
Epoch 4/10
5/5 [==============================] - 0s 8ms/step - loss: 0.1706 - accuracy: 0.5535
Epoch 5/10
5/5 [==============================] - 0s 7ms/step - loss: 0.1678 - accuracy: 0.5660
Epoch 6/10
5/5 [==============================] - 0s 7ms/step - loss: 0.1657 - accuracy: 0.5660
Epoch 7/10
5/5 [==============================] - 0s 7ms/step - loss: 0.1638 - accuracy: 0.5723
Epoch 8/10
5/5 [==============================] - 0s 6ms/step - loss: 0.1619 - accuracy: 0.5849
Epoch 9/10
5/5 [==============================] - 0s 8ms/step - loss: 0.1601 - accuracy: 0.6101
Epoch 10/10
5/5 [==============================] - 0s 7ms/step - loss: 0.1583 - accuracy: 0.6226


 20%|████████████████████████████▊                                                                                                                   | 1/5 [00:02<00:09,  2.31s/it]

...

 60%|██████████████████████████████████████████████████████████████████████████████████████▍                                                         | 3/5 [03:18<02:33, 76.86s/it]

Epoch 1/10
313/313 [==============================] - 2s 6ms/step - loss: -0.4795 - accuracy: 0.9193
Epoch 2/10
313/313 [==============================] - 2s 6ms/step - loss: -0.5077 - accuracy: 0.9237
Epoch 3/10
313/313 [==============================] - 2s 6ms/step - loss: -0.5351 - accuracy: 0.9230
Epoch 4/10
313/313 [==============================] - 2s 6ms/step - loss: -0.5621 - accuracy: 0.9242
Epoch 5/10
313/313 [==============================] - 2s 6ms/step - loss: -0.5889 - accuracy: 0.9235
Epoch 6/10
313/313 [==============================] - 2s 6ms/step - loss: -0.6146 - accuracy: 0.9238
Epoch 7/10
313/313 [==============================] - 2s 6ms/step - loss: -0.6409 - accuracy: 0.9256
Epoch 8/10
313/313 [==============================] - 2s 6ms/step - loss: -0.6665 - accuracy: 0.9232
Epoch 9/10
313/313 [==============================] - 2s 6ms/step - loss: -0.6922 - accuracy: 0.9247
Epoch 10/10
313/313 [==============================] - 2s 7ms/step - loss: -0.7178 - accura

 80%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████▏                            | 4/5 [04:52<01:23, 83.64s/it]

Epoch 1/10
313/313 [==============================] - 2s 7ms/step - loss: -0.6615 - accuracy: 0.9319
Epoch 2/10
313/313 [==============================] - 2s 8ms/step - loss: -0.6842 - accuracy: 0.9316
Epoch 3/10
313/313 [==============================] - 3s 9ms/step - loss: -0.7068 - accuracy: 0.9318
Epoch 4/10
313/313 [==============================] - 3s 8ms/step - loss: -0.7288 - accuracy: 0.9339
Epoch 5/10
313/313 [==============================] - 3s 8ms/step - loss: -0.7510 - accuracy: 0.9331
Epoch 6/10
313/313 [==============================] - 3s 9ms/step - loss: -0.7728 - accuracy: 0.9319
Epoch 7/10
313/313 [==============================] - 3s 9ms/step - loss: -0.7942 - accuracy: 0.9339
Epoch 8/10
313/313 [==============================] - 3s 9ms/step - loss: -0.8157 - accuracy: 0.9344
Epoch 9/10
313/313 [==============================] - 3s 9ms/step - loss: -0.8376 - accuracy: 0.9341
Epoch 10/10
313/313 [==============================] - 3s 9ms/step - loss: -0.8593 - accura

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [06:34<00:00, 78.98s/it]


## 5. Result
With little data (one datapoint) and moderate calculation power (M1 Macbook Pro) the approach is able to identify tokens with different effects on the classification. Tokens with negative effects tend towards a negativ sentiment and vice versa.

In [16]:
effect_df = pd.DataFrame(effects, columns=['word', 'effect'])
effect_df['effect'] = effect_df['effect'].apply(lambda x: x[0])
effect_df[(effect_df['effect'] > 1.5) | (effect_df['effect'] < -1.5)]

,word,effect
0,there,-1.688185
4,make,-2.893196
9,it,1.708110
15,dead;,-1.805970
18,"smith,",1.967181
22,"rodriguez,",1.523124
32,amazing,7.405206
34,flawless,4.487193
36,and,1.763144
39,and,1.715159
